### Разработка рекомендательного алгоритма для читателей библиотеки

Алгоритм будет разработан на основе истории каждого читателя метаданные заголовки книг которые интересны, авторы, издание, год издания. Преобразуем их в признаки, а затем находим похожие книги и произведения на основе этих принзнаков. Рекомендации, основанные на содержании. Наши критерии будут заключаться в том, чтобы рекомендовать книги с такими же характеристиками, которые интересны конкретному читателю, похожее название, автор,издание, ключевые слова, дата выхода и т.д.
 1. Создан общий признак вся история читателя
 2. Удалены стоп слова (на русском и английском)
 3. Лемматизация слов
 4. Метод вычисления схожести читателя вычисляется путем взятия косинуса угла между вектором читателя и вектором документа (использовал TfidfVectorizer)
 5. Для поиска 20 подходящих(схожих) документов (книг) для каждого читатателя использовал NearestNeighbors с метрикой косинус сходства


In [2]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import re
from tqdm import notebook
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
import warnings
import pymorphy2
import random
import string
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
%matplotlib

warnings.filterwarnings("ignore")

Using matplotlib backend: <object object at 0x000002BA2BEBD750>


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Oksana\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
#считывание данных
users = pd.read_csv(r'C:\work\тюмень_чемпионат\train_dataset_train\train/users.csv', sep=';', index_col=None, dtype={'age': str, 'chb': str, 'chit_type': str, 'gender': str})
items = pd.read_csv(r'C:\work\тюмень_чемпионат\train_dataset_train\/train/items.csv', sep=';', index_col=None, dtype={'author': str, 'bbk': str, 'izd': str, 'sys_numb': str, 'title': str, 'year_izd': str})
train_transactions = pd.read_csv(r'C:\work\тюмень_чемпионат\train_dataset_train\train/train_transactions_extended.csv', sep=';', index_col=None, dtype={'chb': str, 'date_1': str, 'is_printed': str, 'is_real': str, 'source': str, 'sys_numb': str, 'type': str})

In [4]:
items.izd=items.izd.replace('[б. и.]','без издательства')
items.izd=items.izd.replace('Э','издательствоэ')
items.izd=items.izd.replace('Б. и.','без издательства')
items.izd=items.izd.replace('Гос. изд-во','государственное издательство')
items.izd=items.izd.replace('Высш. шк.','высшая школа')

In [5]:
#corpus_bbk=[]
#for i in range(len(items)):
   # corpus_bbk.append(items.bbk.loc[i][0])

In [6]:
#items['bbk_kod']=np.array(corpus_bbk)

In [7]:
#bbk_kod_dict = {    'Ш': 'Филологические науки. Художественная литература',
                 #   'Х': 'Право. Юридические науки',
                  #  'Т': 'История. Исторические науки',
                  #  'У': 'Экономика. Экономические науки',
                  #  'Ю': 'Философия и Психология',
                  #  'Ч': 'Культура. Наука. Просвещение',
                  #  'Щ': 'Искусство',
                  #  'Р': 'Здравоохранение. Медицинские науки',
                  #  'С': 'Общественные науки в целом',
                  #  'Э': 'Религия. Мистика. Свободомыслие',
                  #  'З': 'Энергетика и Радиоэлектроника',
                 #   'Е': 'Биологические науки',
                  #  'В': 'Физико-математические науки',
                 #   'Ц': 'Военное дело. Военная наука',
                 ##   'Н': 'Строительство',
                 #   'Д': 'Науки о Земле (геодезические, геофизические,геологические и географические науки)',
                 #   'П': 'Сельское и лесное хозяйство. Сельскохозяйственные и лесохозяйственные науки',
                 #   'Л': 'Химическая технология. Химические производства.Пищевые производства',
                 #   'О': 'Технические науки',
                 #   'М': 'Технология древесины. Производства легкой промышленности. Домоводство. Бытовые услуги. Полиграфическое производство. Фотокинотехника',
                 #   'К': 'Технология металлов. Машиностроение. Приборостроение',
                 #   'Г': 'Химические науки',
                 #   'Ж': 'Техника',
                 #   'Ф': 'Политика. Политическая наука',
                 #   'Я': 'Литература универсального содержания',
                 #   'И': 'Горное дело',
                  #  'А': 'Общенаучное и междисциплинарное знание',
                 #   'Б': 'Естественные науки в целом'
                    
              # }


In [8]:
#items['kod_bbk'] = items['bbk_kod'].apply(lambda x: bbk_kod_dict.get(x, 0))

In [9]:
cyrillic_lower_letters = 'абвгдежзиклмнопрстуфхцчшщ'

def generate_random_string(length):# для генерации строки
    letters = cyrillic_lower_letters
    rand_string = ''.join(random.choice(letters) for i in range(length))
    return rand_string



In [10]:
for x in notebook.tqdm(range(len(items["title"]))):# создадим заголовки книгам без названия 
    if items["title"].iloc[x]=='отсутствует':
        i=generate_random_string(10)
        items["title"].iloc[x]=items["title"].iloc[x].replace('отсутствует', i) 

  0%|          | 0/354355 [00:00<?, ?it/s]

In [11]:
feature_year_izd=items.year_izd.unique().tolist()

In [12]:
items_df=pd.DataFrame()
for x in notebook.tqdm(feature_year_izd):
    items_new=items[items["year_izd"]==x]
    i=generate_random_string(8)
    items_new["year_izd"]=np.array(i) 
    items_df=items_df.append(items_new)

  0%|          | 0/2506 [00:00<?, ?it/s]

In [13]:
x=items_df.year_izd.value_counts().index.tolist()[0]

In [14]:
items_df.year_izd=items_df.year_izd.str.replace(x,'')# заменим пустым значением те ячейки, где отсутствовал год

In [15]:
items=items_df.copy()

In [16]:
#перенесем в историю читателей все данные о книгах
title=dict(zip(items.sys_numb,items.title))
author=dict(zip(items.sys_numb,items.author))
izd=dict(zip(items.sys_numb,items.izd))
year_izd=dict(zip(items.sys_numb,items.year_izd))
age=dict(zip(users.chb,users.age))
gender=dict(zip(users.chb,users.gender))
train_transactions['title'] = train_transactions['sys_numb'].apply(lambda x: title.get(x, 0))
train_transactions['author'] = train_transactions['sys_numb'].apply(lambda x: author.get(x, 0))
train_transactions['izd'] = train_transactions['sys_numb'].apply(lambda x: izd.get(x, 0))
train_transactions['age'] = train_transactions['chb'].apply(lambda x: age.get(x, 0))
train_transactions['gender'] = train_transactions['chb'].apply(lambda x: gender.get(x, 0))
train_transactions['year_izd'] = train_transactions['sys_numb'].apply(lambda x: year_izd.get(x, 0))


In [17]:
train_transactions.head(5)

,chb,sys_numb,date_1,is_real,type,source,is_printed,title,author,izd,age,gender,year_izd
0,100000641403,RSL01004206702,2021-02-21,yes,скачивание,dlib.rsl.ru,False,Придание огнестойкости деревянным постройкам,"Максимов, Владимир Лаврович",Гл. упр. землеустройства и земледелия,55,male,бндгвмжз
1,100000641403,RSL01000769304,2021-03-23,yes,скачивание,dlib.rsl.ru,False,"Медицинская, научная и общественная деятельнос...","Шалаев, Николай Федорович",none,55,male,асриацуе
2,100000641403,RSL01004211574,2021-02-21,yes,скачивание,dlib.rsl.ru,False,Кирпичная изба,"Максимов, Владимир Лаврович",М-во зем.,55,male,змцшжрту
3,100000644359,RSL01009800093,2021-03-16,yes,книговыдача,единый просмоторщик,False,Синтез и свойства новых гетероциклических соед...,"Акылбеков, Нургали Икрамович",none,74,male,ошетщичш
4,100000644359,RSL01003557352,2021-03-10,yes,книговыдача,единый просмоторщик,False,Медали в честь Александра Сергеевича Пушкина,"Ильин, Алексей Алексеевич 1858-1942",тип. А. Бенке,74,male,пбокшеаш


In [18]:
top_10_authors = train_transactions.groupby('author')['title'].count().reset_index().sort_values('title', ascending=False).head(10).set_index('author')
top_10_authors.head(10)

,title
author,
none,49906
отсутствует,19241
Конволют,652
Россия. Законы и постановления,191
"Зверев, Сергей Иванович 1964-",165
"Струнников, Сергей Николаевич 1907-1944",124
"Серова, Марина Сергеевна",102
"Нащокина, Мария Владимировна",91
"Сарабьянов, Дмитрий Владимирович",80


In [19]:
top_10 = train_transactions.groupby('izd')['title'].count().reset_index().sort_values('title', ascending=False).head(10).set_index('izd')
top_10.head(100)

,title
izd,
none,82987
отсутствует,19241
Наука,4562
без издательства,3844
Эксмо,3337
Юрлитинформ,2088
АСТ,1895
"Манн, Иванов и Фербер",1457
Искусство,1333


In [20]:
train_transactions['date_1'] = pd.to_datetime(train_transactions['date_1'])
train_transactions = train_transactions.sort_values(by='date_1')

In [21]:
train_transactions["month"] = pd.to_datetime(train_transactions["date_1"]).dt.month
train_transactions["year"] = pd.to_datetime(train_transactions["date_1"]).dt.year

In [22]:
train_transactions.izd=train_transactions.izd.str.replace('none','')
train_transactions.izd=train_transactions.izd.str.replace('отсутствует','')
items.izd=items.izd.str.replace('none','')
items.izd=items.izd.str.replace('отсутствует','')

In [23]:
train_transactions.author=train_transactions.author.str.replace('none','')
train_transactions.author=train_transactions.author.str.replace('отсутствует','')
items.author=items.author.str.replace('none','')
items.author=items.author.str.replace('отсутствует','')

In [24]:
train_transactions.year_izd=train_transactions.year_izd.str.replace('отсутствует','')
items.year_izd=items.year_izd.str.replace('отсутствует','')

In [25]:
# функция для создания истории читателей: все названия книг, авторы, издания, год издания
def feature_reader (df, n):
    reader=df[df['chb']== n ]
    reader['title']=reader['title'].astype(str)+" "+reader['author'].astype(str)+" "+reader['izd'].astype(str)+" "+reader['year_izd'].astype(str)
    reader_sum=reader['title'].tolist()
    total=' '.join(reader_sum)
    reader_string=[n]+ [total]
    return reader_string

In [26]:
feature_users=users.chb.unique().tolist()

In [27]:
train=[]
for i in notebook.tqdm(feature_users):
     train.append(feature_reader (train_transactions, i))

  0%|          | 0/16753 [00:00<?, ?it/s]

In [28]:
train_data = pd.DataFrame(train, columns = ['chb', 'title'])

In [29]:
train_data.isnull().sum()

chb      0
title    0
dtype: int64

In [30]:
train_data.head(10)

,chb,title
0,300001020830,нвмвдгцлщм Вердикт присяжных заседателей в ...
1,300001113642,Губернаторы и ханы : личностный фактор правово...
2,300001148466,аатзбмшслч Вестник Пермского университета =...
3,300001117011,Создание эмоционально-комфортного состояния мл...
4,200001038094,Архив эпохи : тематическое единство русской фи...
5,200001120504,Ревматоидный артрит с ревматоидными узелками :...
6,300000849123,хфваомшкаф Равновесная сорбция антоцианов н...
7,300001002599,Теоретические проблемы отечественного архивове...
8,200000931895,Педагогическое сопровождение профессионального...
9,300000806816,Татуирование как объект социологического иссле...


In [61]:
morph = pymorphy2.MorphAnalyzer()# Для лемматизациия русских слов
stemmer = WordNetLemmatizer()# Для английских слов

def lemmatize(text):# функция для русских слов
    words = text.split() 
    res=[]
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

### Теперь все текстовые данные приведем к общему виду: 
     1. Удалим пунктуацию, стоп слова, цифры
     2. Переведем в нижний регистр
     3. Лемматизация слов

In [60]:
corpus = train_data['title'].str.lower().values
corpus_reader = []
for i in notebook.tqdm(corpus):
    j=i
    j=re.sub(r"[^A-Za-z]"," ",str(j)).lower()
   
    j = j.lower()
    tokens = j.split()
    tokens = [stemmer.lemmatize(word) for word in tokens]
    j = ' '.join(tokens)

    
    j = nltk.word_tokenize(j)
    j = [word for word in j if not  word in set(stopwords.words("english"))]
    word_eng = " ".join(j)
    
    i=re.sub(r"[^А-Яа-я^]"," ",str(i)).lower()
    
    i = nltk.word_tokenize(i)
    i = [word for word in i if not word in set(stopwords.words("russian"))]
    
   
    test=[]
    for j in i:
        i = " ".join(lemmatize(j))
        test.append(i)
        
    
    word_rus = " ".join(test)   
    string_word = word_rus+' '+ word_eng
    corpus_reader.append(string_word)

  0%|          | 0/16753 [00:00<?, ?it/s]

In [62]:
items['book_string']=items['title'].astype(str)+" "+items['author'].astype(str)+" "+items['izd'].astype(str)+" "+items['year_izd'].astype(str)

In [63]:
corpus = items['book_string'].str.lower().values
corpus_book = []
for i in notebook.tqdm(corpus):
    j=i
    j=re.sub(r"[^A-Za-z]"," ",str(j)).lower()
   
    j = j.lower()
    tokens = j.split()
    tokens = [stemmer.lemmatize(word) for word in tokens]
    j = ' '.join(tokens)

    j = nltk.word_tokenize(j)
    j = [word for word in j if not  word in set(stopwords.words("english"))]
    word_eng = " ".join(j)
    
    i=re.sub(r"[^А-Яа-я^]"," ",str(i)).lower()
    i = nltk.word_tokenize(i)
    
    i = [word for word in i if not word in set(stopwords.words("russian"))]
    test=[]
    for j in i:
        i = " ".join(lemmatize(j))
        test.append(i)
        
    word_rus = " ".join(test)   
    string_word = word_rus+' '+ word_eng
    corpus_book.append(string_word)

  0%|          | 0/354354 [00:00<?, ?it/s]

In [64]:
items['title_book']=np.array(corpus_book)

In [65]:
vec = TfidfVectorizer()
mx_books = vec.fit_transform(items['title_book'])
mx_reader = vec.transform([corpus_reader[0]])   

mx_books.shape, mx_reader.shape

((354354, 201573), (1, 201573))

In [66]:
df = pd.DataFrame(mx_books[25].T.todense(), index=vec.get_feature_names(), columns=["TF-IDF"] ) 
df = df.sort_values('TF-IDF', ascending=False) 
print (df.head(15))

                TF-IDF
фашизм        0.393924
моисеев       0.382459
доктрина      0.362892
итальянский   0.341059
становление   0.271736
философия     0.254861
философский   0.244003
политический  0.229165
дмитрий       0.229005
сергеевич     0.202886
развитие      0.188735
угхдлкина     0.175748
диссертация   0.124872
кандидат      0.116332
наука         0.102214


In [67]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine',algorithm='brute')

In [68]:
model_knn.fit(mx_books)

NearestNeighbors(algorithm='brute', metric='cosine')

In [69]:
query_index = np.random.choice(items.shape[0])

In [71]:
mx_reader = vec.transform([corpus_reader[0]])
distances, indices = model_knn.kneighbors(mx_reader,n_neighbors=20)
rec=items.iloc[indices.flatten()]
rec['chb'] = np.array(train_data['chb'].loc[0])
rec['neighbors'] = distances.flatten()
rec=rec.sort_values(by='neighbors', ascending=False)
rec.head(7)

,sys_numb,title,author,izd,year_izd,bbk,book_string,title_book,chb,neighbors
95609,RSL01000292972,Дифференциация уголовной ответственности и нак...,"Васильевский, Александр Валентинович",,валдууси,"Х628.102,0",Дифференциация уголовной ответственности и нак...,дифференциация уголовный ответственность наказ...,300001020830,0.579899
7526,RSL01008594826,Качество уголовного закона. Проблемы Общей час...,"Рарог, Алексей Иванович",,угхдлкин,none,Качество уголовного закона. Проблемы Общей час...,качество уголовный закон проблема общий часть ...,300001020830,0.579566
152808,RSL01004389874,Правовая помощь по уголовным делам как комплек...,"Лазутин, Лев Александрович",,шдтщохрх,"Х916.8,0",Правовая помощь по уголовным делам как комплек...,правовой помощь уголовный дело комплексный фор...,300001020830,0.575660
177812,RSL01008740429,Актуальные проблемы сравнительного и междунаро...,,МГИМО-Ун-т,угхдлкин,Х91я43\nХ99я43,Актуальные проблемы сравнительного и междунаро...,актуальный проблема сравнительный международны...,300001020830,0.568766
10660,RSL01008263271,Адвокат в уголовном процессе : учебное пособие...,,Закон и право,угхдлкин,Х629.316я73-1,Адвокат в уголовном процессе : учебное пособие...,адвокат уголовный процесс учебный пособие студ...,300001020830,0.562708
184128,RSL01008598319,Качество уголовного закона. Проблемы Общей час...,,Проспект,угхдлкин,"Х628.10,0",Качество уголовного закона. Проблемы Общей час...,качество уголовный закон проблема общий часть ...,300001020830,0.561590
70416,RSL01003237205,Влияние международного уголовного права на рос...,"Кибальник, Алексей Григорьевич",,жнбтлрлж,"Х916.8,0",Влияние международного уголовного права на рос...,влияние международный уголовный право российск...,300001020830,0.553530


In [73]:
data=pd.DataFrame()
for x in notebook.tqdm(range(len(train_data))):
    mx_reader = vec.transform([corpus_reader[x]])
    

    distances, indices = model_knn.kneighbors(mx_reader,n_neighbors=20)
    rec=items.iloc[indices.flatten()]
    rec['chb'] = np.array(train_data['chb'].loc[x])

    rec['neighbors'] = distances.flatten()
    rec=rec.sort_values(by='neighbors', ascending=False)
    data=data.append(rec)
    

  0%|          | 0/16753 [00:00<?, ?it/s]

In [74]:
data

,sys_numb,title,author,izd,year_izd,bbk,book_string,title_book,chb,neighbors
95609,RSL01000292972,Дифференциация уголовной ответственности и нак...,"Васильевский, Александр Валентинович",,валдууси,"Х628.102,0",Дифференциация уголовной ответственности и нак...,дифференциация уголовный ответственность наказ...,300001020830,0.579899
7526,RSL01008594826,Качество уголовного закона. Проблемы Общей час...,"Рарог, Алексей Иванович",,угхдлкин,none,Качество уголовного закона. Проблемы Общей час...,качество уголовный закон проблема общий часть ...,300001020830,0.579566
152808,RSL01004389874,Правовая помощь по уголовным делам как комплек...,"Лазутин, Лев Александрович",,шдтщохрх,"Х916.8,0",Правовая помощь по уголовным делам как комплек...,правовой помощь уголовный дело комплексный фор...,300001020830,0.575660
177812,RSL01008740429,Актуальные проблемы сравнительного и междунаро...,,МГИМО-Ун-т,угхдлкин,Х91я43\nХ99я43,Актуальные проблемы сравнительного и междунаро...,актуальный проблема сравнительный международны...,300001020830,0.568766
10660,RSL01008263271,Адвокат в уголовном процессе : учебное пособие...,,Закон и право,угхдлкин,Х629.316я73-1,Адвокат в уголовном процессе : учебное пособие...,адвокат уголовный процесс учебный пособие студ...,300001020830,0.562708
...,...,...,...,...,...,...,...,...,...,...
247680,RSL01001559332,Англо-русский словарь глагольных словосочетаний,,,пбвкцжса,none,Англо-русский словарь глагольных словосочетани...,ангнуть русский словарь глагольный словосочета...,300001155049,0.558695
353907,RSL01007283516,Сложные глагольные словосочетания в современно...,"Богачев, Юрий Павлович",без издательства,еесдцичи,Ш141.2-22,Сложные глагольные словосочетания в современно...,сложный глагольный словосочетание современный ...,300001155049,0.493118
353908,RSL01001700089,Функционально-коммуникативная типология устойч...,"Воробьева, Галина Константиновна",Ведас,клиехнтш,none,Функционально-коммуникативная типология устойч...,функционально коммуникативный типология устойч...,300001155049,0.391879
353911,RSL01008443560,Глагольные словосочетания с родительным падежо...,"Солоницын, Ю. В.",без издательства,кдптцбщу,none,Глагольные словосочетания с родительным падежо...,глагольный словосочетание родительный падеж со...,300001155049,0.204285


In [75]:
submission = pd.DataFrame({
    "chb" : data["chb"].values,
    "sys_numb" : data["sys_numb"].values
}) 

In [76]:
submission.to_csv("submission.csv", index=False, sep=';')